In [ ]:

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from keras.datasets import cifar100
from keras.utils import np_utils



In [ ]:
# 학습용 데이터 , test용 데이터 수집
(x_train,y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

169009152/169001437 [==============================] - 3s 0us/step


In [ ]:
# 데이터의 형태 확인
x_train.shape ,y_train.shape

((50000, 32, 32, 3), (50000, 1))

In [ ]:
y_train = np_utils.to_categorical(y_train, 100)
y_test = np_utils.to_categorical(y_test, 100)

In [ ]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#pretrained 된 resnet50 모델의 배치 정규화 층은 학습이 가능하게 둬야한다 기존 사전학습된 배치정규화층은 imagenet 학습용 데이터로 사전 학습, cifar100 test data가 새로 들어오면 문제발생
#기존 layer가  imagenet 데이터의 통계적인 분포를 따르기 떄문
# 그렇기 때문에 기존 resnet50모델의 배채정규화 layer도 학습 가능하게 , resnet모델의 나머지 층은 학습 불가능하게 사전 학습된거 사용하기 위해서 
for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False



model = Sequential()
#upsampling을 통한 이미지 확대 후 학습 방법 32-> 224로 학습하는 성능 유지 가능
#이미지 크기를 키우는 과정을 대체
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
# 이미지를 키운 후 resnet model 적용
model.add(resnet_model)
# 새로운 학습시켜야할 모델 생성 resnet에 추가 
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(100, activation='softmax'))


94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
#너무 오래걸려서 epoch 7일때 중단
model.fit(x_train,y_train,batch_size=32 , epochs=12 , validation_data=(x_test,y_test))

Epoch 1/12
1563/1563 [==============================] - 623s 399ms/step - loss: 2.0187 - accuracy: 0.4822 - val_loss: 1.1632 - val_accuracy: 0.6612
Epoch 2/12
1563/1563 [==============================] - 626s 401ms/step - loss: 1.1873 - accuracy: 0.6549 - val_loss: 0.9386 - val_accuracy: 0.7177
Epoch 3/12
1563/1563 [==============================] - 627s 401ms/step - loss: 0.9981 - accuracy: 0.7041 - val_loss: 0.8715 - val_accuracy: 0.7387
Epoch 4/12
1563/1563 [==============================] - 628s 402ms/step - loss: 0.8870 - accuracy: 0.7343 - val_loss: 0.8576 - val_accuracy: 0.7445
Epoch 5/12
1563/1563 [==============================] - 628s 402ms/step - loss: 0.7975 - accuracy: 0.7568 - val_loss: 0.8286 - val_accuracy: 0.7516
Epoch 6/12
1563/1563 [==============================] - 630s 403ms/step - loss: 0.7284 - accuracy: 0.7750 - val_loss: 0.7941 - val_accuracy: 0.7632
Epoch 7/12
 140/1563 [=>............................] - ETA: 8:38 - loss: 0.6433 - accuracy: 0.7935

KeyboardInterrupt: ignored

In [ ]:
model.fit(x_train,y_train,batch_size=32 , epochs=2 , validation_data=(x_test,y_test))

Epoch 1/2
 902/1563 [================>.............] - ETA: 3:57 - loss: 0.6490 - accuracy: 0.7961

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(x_test,y_test,batch_size=32)

313/313 [==============================] - 60s 192ms/step - loss: 0.8110 - accuracy: 0.7597


[0.8109832406044006, 0.7597000002861023]

In [ ]:
model.summary()
#학습 trainable parameter => resnet의 batch-norm layer, resnet 모델 이후로 추가 한 layer parameter

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d (UpSampling2D) (None, 64, 64, 3)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 128, 128, 3)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 256, 256, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0

In [ ]:
model.layers